<a href="https://colab.research.google.com/github/asattiraju13/wildfire-detection/blob/analysis/Fire_Confidence_Predictor_%2B_Australia_Map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/drive')
train_path = 'drive/MyDrive/Colab_Notebooks/fire_confidence_predictor.csv'

Mounted at /content/drive


In [15]:
df = pd.read_csv(train_path)

In [16]:
df

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
0,-11.8070,142.0583,313.0,1.0,1.0,2019-08-01,56,Terra,MODIS,48,6.3,297.3,6.6,D,0
1,-11.7924,142.0850,319.3,1.0,1.0,2019-08-01,56,Terra,MODIS,71,6.3,297.3,11.3,D,0
2,-12.8398,132.8744,311.6,3.1,1.7,2019-08-01,57,Terra,MODIS,42,6.3,298.7,23.1,D,0
3,-14.4306,143.3035,310.1,1.1,1.1,2019-08-01,57,Terra,MODIS,33,6.3,296.1,6.5,D,0
4,-12.4953,131.4897,310.3,4.0,1.9,2019-08-01,57,Terra,MODIS,36,6.3,298.8,27.6,D,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36006,-28.0656,131.6334,312.7,1.2,1.1,2019-09-30,1644,Aqua,MODIS,85,6.3,292.9,14.2,N,0
36007,-28.2625,128.5494,304.9,1.0,1.0,2019-09-30,1644,Aqua,MODIS,27,6.3,290.9,6.4,N,0
36008,-28.2941,128.5123,301.7,1.0,1.0,2019-09-30,1644,Aqua,MODIS,43,6.3,290.4,5.5,N,0
36009,-28.3001,128.5530,303.6,1.0,1.0,2019-09-30,1644,Aqua,MODIS,41,6.3,290.3,5.7,N,0


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36011 entries, 0 to 36010
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   latitude    36011 non-null  float64
 1   longitude   36011 non-null  float64
 2   brightness  36011 non-null  float64
 3   scan        36011 non-null  float64
 4   track       36011 non-null  float64
 5   acq_date    36011 non-null  object 
 6   acq_time    36011 non-null  int64  
 7   satellite   36011 non-null  object 
 8   instrument  36011 non-null  object 
 9   confidence  36011 non-null  int64  
 10  version     36011 non-null  float64
 11  bright_t31  36011 non-null  float64
 12  frp         36011 non-null  float64
 13  daynight    36011 non-null  object 
 14  type        36011 non-null  int64  
dtypes: float64(8), int64(3), object(4)
memory usage: 4.1+ MB


In [18]:
df.acq_date = pd.to_datetime(df['acq_date'])
df = df.drop('acq_time',axis=1)

In [ ]:
df.drop(['instrument','type','version','scan','track','acq_time'],axis=1,inplace=True)

In [19]:
df['day'] = df.acq_date.dt.day
df['month'] = df.acq_date.dt.month

df.drop(['acq_date'],axis=1,inplace=True)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36011 entries, 0 to 36010
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   latitude    36011 non-null  float64
 1   longitude   36011 non-null  float64
 2   brightness  36011 non-null  float64
 3   scan        36011 non-null  float64
 4   track       36011 non-null  float64
 5   satellite   36011 non-null  object 
 6   instrument  36011 non-null  object 
 7   confidence  36011 non-null  int64  
 8   version     36011 non-null  float64
 9   bright_t31  36011 non-null  float64
 10  frp         36011 non-null  float64
 11  daynight    36011 non-null  object 
 12  type        36011 non-null  int64  
 13  day         36011 non-null  int64  
 14  month       36011 non-null  int64  
dtypes: float64(8), int64(4), object(3)
memory usage: 4.1+ MB


In [24]:
df.head()

,latitude,longitude,brightness,scan,track,satellite,confidence,version,bright_t31,frp,daynight,type,day,month
0,-11.8070,142.0583,313.0,1.0,1.0,0,48,6.3,297.3,6.6,0,0,1,8
1,-11.7924,142.0850,319.3,1.0,1.0,0,71,6.3,297.3,11.3,0,0,1,8
2,-12.8398,132.8744,311.6,3.1,1.7,0,42,6.3,298.7,23.1,0,0,1,8
3,-14.4306,143.3035,310.1,1.1,1.1,0,33,6.3,296.1,6.5,0,0,1,8
4,-12.4953,131.4897,310.3,4.0,1.9,0,36,6.3,298.8,27.6,0,0,1,8


In [22]:
df.instrument.value_counts()

MODIS    36011
Name: instrument, dtype: int64

In [23]:
df['daynight'].replace({'D':0,'N':1},inplace=True)
df['satellite'].replace({'Terra':0,'Aqua':1},inplace=True)

df = df.drop('instrument', axis = 1)

In [25]:
X = df.drop('confidence',axis=1)
y = df.confidence

In [26]:
len(df) * 0.7

25207.699999999997

In [27]:
X_train = X[:25207]
X_test = X[25207:]

y_train = y[:25207]
y_test = y[25207:]

In [32]:
from catboost import CatBoostRegressor

In [29]:
from xgboost import XGBRegressor

In [33]:
from sklearn.metrics import mean_squared_error

In [34]:
cat = CatBoostRegressor(loss_function='RMSE',learning_rate=0.10)
xgb = XGBRegressor(n_estimators = 500, learning_rate = 0.05, max_depth = 6)

In [35]:
cat.fit(X_train, y_train)

0:	learn: 21.7722656	total: 57.9ms	remaining: 57.9s
1:	learn: 20.8080155	total: 63.7ms	remaining: 31.8s
2:	learn: 19.9969839	total: 69.4ms	remaining: 23.1s
3:	learn: 19.2533268	total: 74.7ms	remaining: 18.6s
4:	learn: 18.6298875	total: 79.8ms	remaining: 15.9s
5:	learn: 18.0232391	total: 84.7ms	remaining: 14s
6:	learn: 17.5140100	total: 89.7ms	remaining: 12.7s
7:	learn: 17.0978119	total: 95ms	remaining: 11.8s
8:	learn: 16.7399829	total: 100ms	remaining: 11.1s
9:	learn: 16.4483518	total: 106ms	remaining: 10.5s
10:	learn: 16.1587256	total: 111ms	remaining: 9.97s
11:	learn: 15.9313240	total: 116ms	remaining: 9.53s
12:	learn: 15.7170139	total: 121ms	remaining: 9.2s
13:	learn: 15.5340672	total: 126ms	remaining: 8.88s
14:	learn: 15.3861255	total: 131ms	remaining: 8.61s
15:	learn: 15.2498600	total: 136ms	remaining: 8.38s
16:	learn: 15.1161564	total: 141ms	remaining: 8.17s
17:	learn: 15.0021621	total: 146ms	remaining: 7.99s
18:	learn: 14.9010582	total: 152ms	remaining: 7.82s
19:	learn: 14.78321

In [36]:
np.sqrt(mean_squared_error(y_test,cat.predict(X_test)))

14.457172403497458

In [37]:
xgb.fit(X_train, y_train)

[17:35:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(learning_rate=0.05, max_depth=6, n_estimators=500)

In [38]:
np.sqrt(mean_squared_error(y_test,xgb.predict(X_test)))

14.741964169598749

In [39]:
y_pred = cat.predict(X_test)

In [40]:
lat = pd.Series(X_test.latitude)

In [41]:
long = pd.Series(X_test.longitude)

In [42]:
pred = pd.Series(y_pred)

In [43]:
actual = pd.Series(y_test)

In [44]:
map_df = pd.concat([lat,long,actual],axis=1)

In [45]:
map_df = map_df.reset_index().drop('index',axis=1)

In [46]:
map_df = pd.concat([map_df, pred], axis=1)

In [47]:
map_df = map_df.rename(columns={0:'pred'})

In [48]:
print(map_df.head())
print(map_df.shape)

   latitude  longitude  confidence       pred
0  -28.9321   152.3562          55  51.612934
1  -28.9335   152.3453          82  70.962837
2  -26.4043   150.6507          10  33.296298
3  -26.4145   150.9770          42  48.893853
4  -26.4242   150.9785           8  40.677504
(10804, 4)


In [51]:
import plotly.express as px

fig = px.scatter_geo(map_df,lat='latitude',lon='longitude',color='confidence')
fig.update_layout(title = 'Australia Map', title_x=0.5)
fig.show()

In [31]:
!pip install catboost

     |████████████████████████████████| 76.3 MB 1.2 MB/s 
